In [10]:
pip install -q -U google-genai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from dotenv import load_dotenv
import minsearch
import json

In [13]:
with open('01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [17]:
index.fit(documents)

In [18]:
from google import genai

# Load environment variables from .env file
load_dotenv()

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.



In [19]:
q = 'the course has already started, can I still enroll?'

In [20]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=q
)
print(response.text)

Whether you can still enroll in a course that has already started depends on several factors:

*   **The course's policy:** Many courses have a deadline for enrollment, especially those with a fixed start and end date. Some courses may allow late enrollment, perhaps with a penalty or an understanding that you'll need to catch up.
*   **The type of course:** Self-paced courses or those with rolling enrollment are more likely to allow you to join after the official start date.
*   **The instructor's or institution's policy:** Sometimes, you can contact the instructor or the institution offering the course to request special permission to enroll late. They might consider your circumstances and decide whether it's feasible for you to catch up and succeed.
*   **Availability:** If there are physical limitations on the course due to space constraints or kit requirements, a late enrolment might not be possible.

To find out if you can still enroll, take these steps:

1.  **Check the course we

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    return response.text

In [24]:
def rag(query):
    search_results = search(query)
    
    if not search_results:
        return "Sorry, I couldn't find any relevant information."
    
    prompt = build_prompt(query, search_results)
    
    answer = llm(prompt)
    
    return answer

In [25]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n"

In [26]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [43]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
pip install "elasticsearch>=8.0.0,<9.0.0"

In [49]:
from elasticsearch import Elasticsearch

In [50]:
es_client = Elasticsearch('http://localhost:9200')

In [52]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)